# Алгоритм Витерби для чешского корпуса

Импортируем библиотеки

In [77]:
import conllu # парсилка корпуса
from io import open
import pickle # для сохранения состояния обучения
import numpy as np

In [78]:
dir_in = "./UD_Czech-PDT/"
train_file = "cs_pdt-ud-train-l.conllu"
test_file = "cs_pdt-ud-test.conllu"

Открываем файл с тренировочным корпусом

In [79]:
data_train = conllu.parse_incr(open(dir_in+train_file,'r',encoding = "utf-8"))

Подсчитываем количество триграммов,биграммов и монограммов в корпусе,используя словари

In [80]:
c3 = {} # количество различных триграммов 
c2 = {} # //---//---// биграммов
c1 = {} # //---//---// монограммов
cT = {} # //---//---// пар слово-тег
for tokenlist in data_train:
    sentence = [["*","*"]] + [["*","*"]] + [[tokenlist[i]["lemma"], tokenlist[i]["upostag"]]
        for i in range(len(tokenlist)) ] + [["STOP","STOP"]]

    for i in range(0, len(sentence)-2):
        c3old = c3.get( (sentence[i][1], sentence[i+1][1], sentence[i+2][1]), 0)# если такого триграмма еще нет в словаре,вернуть 0
        c3.update( { (sentence[i][1], sentence[i+1][1], sentence[i+2][1]): c3old+1 } ) #если есть,увеличиваем количество на 1

    for i in range(0, len(sentence)-1):
        c2old = c2.get( (sentence[i][1], sentence[i+1][1]), 0)
        c2.update( { (sentence[i][1], sentence[i+1][1]): c2old+1 } )

    for i in range(0, len(sentence)):
        c1old = c1.get( (sentence[i][1]), 0)
        c1.update( { (sentence[i][1]): c1old+1 } )

    for i in range(0, len(sentence)):
        cTold = cT.get( (sentence[i][1], sentence[i][0]), 0)
        cT.update( { (sentence[i][1], sentence[i][0]): cTold+1 } )


Сохраняем данные в пикл, чтобы потом использовать, не обучая заново

In [81]:
print (len(c3))
print (len(c2))
print (len(c1))
print (len(cT))

# Saving:
with open('cs_czech.pkl', 'wb') as f:
    pickle.dump([cT, c1, c2, c3], f)


3050
275
20
40242


Загружаем данные из файла обратно

In [82]:
# Getting back the cs:
with open('cs_czech.pkl', 'rb') as f:
    cT, c1, c2, c3 = pickle.load(f)

In [83]:
tags = list(c1.keys())# список уникальных тегов
tags.remove('*')
tags.remove('STOP')
k = len(tags) # количество уникальных тегов

Вспомогательные функции из алгоритма Витерби

In [84]:
def q(s, u, v):
    eps = 1e-7
    return c3.get((u, v, s), eps) / c2.get((u, v), eps)


def e(x, s):
    eps = 1e-7
    return cT.get((s, x), eps) / c1.get((s), eps)


def K(n): # выдает возможные тэги в зависимости от положения слова в предложении (в начале предложения - *) 
    if n == -1 or n == 0:
        return ['*']
    else:
        return tags

print(K(10))


['PART', 'NUM', 'VERB', 'ADV', 'ADP', 'NOUN', 'CCONJ', 'PROPN', 'PRON', 'DET', 'PUNCT', 'SYM', 'INTJ', '_', 'SCONJ', 'X', 'ADJ', 'AUX']


Сам алгоритм Витерби в виде функции от токенизированного предложения ,возвращающей список тегов

In [85]:
def viterbi(sent):
    Pi = {(0, '*', '*'): 1}
    bp = {}
    n = len(sent) - 2
    y = [""] * (n + 1)

    for k in range(1, n + 1):
        xk = sent[k]
        print(xk)
        for u in K(k-1):
            for v in K(k):
                w = K(k-2)
                v1 = map(lambda wi:
                            Pi.get((k-1, wi, u)) *
                            q(v, wi, u) *
                            e(xk, v), w)
                v1 = list(v1)
                PiNew = max(v1)
                bpNew = w[v1.index(PiNew)]
                Pi.update({(k, u, v): PiNew})
                bp.update({(k, u, v): bpNew})

    v2 = {}
    for u in K(n-1):
        for v in K(n):
            v2.update({(n, u, v): Pi.get((n, u, v)) * q("STOP", u, v)})

    v2max = max(list(v2.values()))
    for (n, u, v), val in v2.items():
        if val == v2max:
            (y[n-1], y[n]) = (u,v)

    for k in range(n-2, 0, -1):
        y[k] = bp.get((k + 2, y[k+1], y[k+2]))

    #print(y[1:])
    return y[1:]


Открываем тестовый корпус и прогоняем по нему алгоритм

In [86]:
data_test = conllu.parse_incr(open(dir_in+test_file,'r',encoding = "utf-8"))
test_tags = [] # оригинальные теги корпуса
predict_tags = [] # предсказанные алгоритмом

for tokenlist in data_test:
    sentence=["*"] + [tokenlist[i]["lemma"] for i in range(len(tokenlist)) ] + ["STOP"]
    test_tags.append([tokenlist[i]["upostag"] for i in range(len(tokenlist)) ])
    predict_tags.append(viterbi(sentence))

štoček
:
chtít
být
vědět
o
hmotnost
pečivo
opakovaně
narážet
na
připomínka
,
že
chléb
a
pečivo
mít
malý
hmotnost
.
dodavatel
reklamace
uznávat
s
ten
,
že
norma
hmotnost
předepisovat
.
mít
pravda
?
Václav
janek
,
valašský
meziříčí
brzký
ustanovení
norma
,
který
přímo
určovat
,
v
který
hmotnost
se
chléb
a
ostatní
pečivo
moci
vyrábět
,
už
platit
.
s
účinnost
od
1
.
8
.
1992
být
provedený
změna
společný
ustanovení
závazný
ČSN
pro
chléb
(
ČSN
56
1001
)
,
pšeničný
pečivo
běžný
(
ČSN
56
1030
)
a
pšeničný
pečivo
jemný
(
ČSN
56
1003
)
.
podle
on
mít
výrobce
možnost
stanovit
hmotnost
výrobek
dohoda
s
odběratel
.
hmotnost
chléb
různý
výrobce
být
značně
rozdílný
a
u
pečivo
být
situace
podobný
.
výrobce
deklarovat
hmotnost
výrobek
v
nabídkový
list
a
na
dodací
list
.
u
balený
výrobek
muset
být
hmotnost
deklarovaný
na
obal
.
skutečnost
,
zda
hmotnost
(
a
kvalita
)
zboží
odpovídat
ten
,
co
být
deklarovaný
výrobce
,
být
mít
prodejce
ověřit
při
přejímka
,
odpovídající
zboží
odmítnout
,
nebo
na
on
při
pr

snižovat
na
5500
a
málo
.
nakonec
Tatra
vyrobit
za
rok
1993
pouze
2950
nový
automobil
a
dohromady
s
vozidlo
z
starý
zásoba
prodat
jen
4048
vůz
.
hospodářský
výsledek
za
minulý
rok
skončit
bilanční
ztráta
1.2
miliarda
koruna
.
daleko
pokračovat
propouštění
zaměstnanec
,
ten
však
signalizovat
americký
manažer
už
předem
.
jako
jediný
výrazný
úspěch
lze
zaznamenat
snad
jen
vítězství
Tatra
v
soutěž
kamión
a
šestý
místo
v
celkový
klasifikace
v
rallye
Paříž
-
Dakar
.
jinak
se
šířit
názor
,
že
Američan
se
moc
dařit
.
znovu
se
objevovat
hlas
,
že
Tatra
hrozit
bankrot
.
informace
od
pramen
jak
být
ten
v
skutečnost
?
nemocný
podnik
vyžadovat
rázný
opatření
,
který
muset
přinést
hned
výsledek
.
naopak
někdy
docházet
ještě
k
snížení
výkon
,
než
se
organismus
uzdravit
a
začít
se
rozjíždět
do
potřebný
obrátka
.
být
ten
případ
Tatra
,
anebo
být
něco
pravda
na
zmíněný
pochybnost
?
místo
spekulace
být
se
vypravit
do
Tatra
a
o
odpověď
na
četný
otázka
být
požádat
rovnou
Gerald
Greenwald
.
číst
na
strana
.

stále
minimum
nutný
k
přežití
.
kolik
stát
Strahov
?
na
Strahov
lze
uspořádat
koncert
zahraniční
kapela
říci
za
350000
USD
,
ten
být
asi
10.5
milión
koruna
.
produkce
Strahov
moci
stát
málo
než
15
milión
koruna
.
reálný
náklad
být
činit
po
zahrnutí
všechen
faktor
26
milión
koruna
.
_
aby
být
nula
od
nula
pojít
,
muset
být
přijít
52000
člověk
při
cena
lístek
500
koruna
.
protože
se
za
normální
návštěvnost
považovat
25000
člověk
,
dostat
se
s
cena
lístek
okamžitě
na
1040
koruna
.
moci
producent
vydělat
?
být
tento
cena
vůbec
přijatelný
?
určitě
zatím
ne
.
superdobrý
kšeft
ovlivňovat
jednoznačně
počasí
.
obvykle
se
přímo
na
stadión
prodat
30
-
40
%
lístek
v
vztah
k
množství
prodaný
lístek
prostřednictví
předprodej
.
při
zásah
počasí
lze
také
prodělat
5000000
koruna
.
Rolling
Stones
v
tento
případ
se
jednat
o
zcela
zvláštní
situace
.
nový
epocha
,
nadšení
člověk
,
masivní
podpora
médium
,
účast
prezident
na
koncert
,
platit
se
autorský
právo
a
hodně
další
správní
poplatek
,
apod
.
vstupenk

dostávat
bez
český
návod
i
další
lék
,
jako
například
tableta
Olfen
-
25
.
i
když
lékař
zároveň
s
lék
předepsat
,
jak
on
užívat
,
být
právo
pacient
rychle
a
srozumitelně
se
dovědět
například
takový
údaj
,
jako
být
kontraindikace
či
možný
vedlejší
účinek
(
třeba
ospalost
a
tudíž
vhodnost
řídit
při
léčba
motorový
vozidlo
)
.
Oskar
firma
pučálka
,
za
bezplatný
instalace
speciální
protisluneční
fólie
Solarmax
na
oddělení
transplantační
jednotka
kostní
dřeň
hematologický
oddělení
II
.
dětský
klinika
fakultní
nemocnice
v
Motol
.
tento
fólie
výrazně
zlepšovat
prostředí
dětský
pacient
.
hodnota
odvedený
dílo
dosáhnout
téměř
100000
koruna
.
kontakt
:
firma
pučálka
,
Malkovský
584
,
190
00
Praha
9
,
telefon
.
+
fax
:
(
02
)
895292
.
student
vědět
,
co
chtít
říkat
se
,
že
za
špatný
ekonomika
moci
ekonom
.
polemizovat
nyní
s
tento
tvrzení
a
ptát
se
,
jaký
být
ten
,
který
vycházet
z
škola
.
odpovídat
Jan
Seger
,
který
být
24
.
únor
1994
v
betlémský
kaple
slavnostně
inaugurovaný
rektor
pražský
VŠE
.

.
o
každý
případ
rozhodovat
ministerstvo
pro
imigrace
.
posuzování
být
nyní
poměrně
přísný
,
protože
míra
nezaměstnanost
v
Kanada
přesáhnout
11
%
.
kontakt
:
kanadský
velvyslanectví
,
obchodní
oddělení
,
na
Petynka
120
,
169
00
Praha
6
,
telefon
.
:
(
02
)
35
79662
,
fax
:
(
02
)
35
54
76
.
byznys
století
(
strana
.
3
)
krádež
a
legální
vývoz
starožitný
předmět
,
ten
být
dobře
zorganizovaný
obchod
.
kdo
vědět
,
o
kolik
být
chudý
.
koupat
se
rád
?
(
strana
.
8
)
hygiena
být
samozřejmý
,
koupání
v
vana
-
pohoda
-
já
přijít
ročně
na
slušný
peníze
.
pokračovat
v
cyklus
domácnost
spořit
.
rozpad
ČSFR
oko
francouzský
týdeník
L
'
Expansion
nahlížet
na
ekonomický
situace
v
ČR
a
SR
rok
po
rozdělení
.
profit
rádce
vůně
v
obchod
využití
,
případně
zneužití
vůně
v
obchodování
.
vůně
jako
nástroj
podnikatel
.
profit
speciál
pobočkový
TÚ
jaký
služba
nabízet
.
dělení
podle
kapacita
.
sledování
provozní
náklad
.
postup
při
výstavba
.
vítěz
blahopřát
v
závěr
loňský
rok
vyhlásit
profit
společně
s
fond
Y

...

přistoupit
k
histologický
a
sérologický
vyšetření
objekt
.
submikroskopický
a
sérologický
vyšetření
ukázat
,
že
"
jazyk
svatý
.
Jan
Nepomucký
"
být
sporně
organický
původ
.
hmota
tvořící
základní
materiál
obsahovat
žádný
histologicky
definovatelný
struktura
,
být
však
barvitelný
a
při
submikroskopický
vyšetření
v
elektronový
mikroskop
být
zjištěný
drobný
vlákénko
,
velký
hrudka
a
blanitý
útvar
.
relikvie
tvořit
dekomponovaný
suchý
mozkový
tkáň
,
což
lze
odvodit
z
makroskopický
i
mikroskopický
podobnost
s
vzorek
hmota
z
nitro
lebka
kníže
Vratislav
a
muž
z
15
.
století
.
tento
závěr
podporovat
pruhovitý
zabarvení
na
vnitřní
strana
mozkovna
,
který
jednak
ukazovat
původní
hladina
mozkový
kaše
,
vzniklý
při
rozpad
mozkový
tkáň
hniloba
,
jednak
já
informovat
o
poloha
hlava
zemřelý
v
hrob
.
tento
hustý
kašovitý
hmota
vyplňovat
určitý
část
dutina
lebeční
.
postup
doba
docházet
k
jeho
vysychání
a
k
snižování
jeho
hladina
.
až
v
doba
exhumace
Jan
z
Pomuk
zůstat
v
mozkovna
jen
jeho
malý
množství

muset
již
dodávat
,
v
který
svůj
bod
být
přímka
s
souběžka
přímka
p
a
v
který
bod
být
míněný
směr
souběžnost
s
přímka
p
.
tvrzení
2
.
(
a
)
být
p
,
q
různý
přímka
,
být
u
přímka
protínající
přímka
p
v
bod
p
a
přímka
q
v
bod
q
,
kde
p
,
q
být
různý
bod
.
nechť
součet
vnitřní
úhel
,
který
svírat
přímka
p
,
q
s
přímka
u
po
jeho
jeden
strana
být
rovný
dva
úhel
pravý
.
potom
lze
vytvořit
společný
kolmice
přímka
p
,
q
.
(
b
)
být
q
souběžka
přímka
p
.
potom
přímka
p
,
q
moci
mít
žádný
společný
kolmice
.
důkaz
.
(
a
)
být
střed
úsečka
p
,
q
,
být
a
bod
,
jenž
být
pata
kolmice
spuštěný
z
bod
s
na
přímka
p
,
být
b
bod
ležící
na
přímka
q
na
druhý
strana
přímka
u
než
bod
a
,
přičemž
úsečka
AP
,
BQ
být
stejně
dlouhý
.
protože
úhel
APS
,
SQB
být
stejně
veliký
,
být
trojúhelník
SPA
,
SQB
shodný
.
přímka
SB
být
tedy
kolmý
na
přímka
q
.
protože
také
úhel
QSB
,
PSA
být
stejně
veliký
,
ležet
bod
a
,
s
,
b
na
přímka
.
přímka
AB
být
tedy
společný
kolmice
přímka
p
,
q
(
viz
obrázek
.
12
)
.
(
b
)
být
r
bod

.
být
p
,
s
,
q
pata
kolmice
spuštěný
z
bod
p
,
s
,
q
na
přímka
q
.
být
-
li
úsečka
page
,
QQ
stejně
dlouhý
,
být
PQQP
Saccheri
čtyřúhelník
a
podle
tvrzení
o
souměrnost
tento
čtyřúhelník
být
spojnice
střed
úsečka
PQ
,
PQ
kolmý
na
přímka
PQ
,
PQ
.
nechť
tedy
například
úsečka
QQ
mít
malý
délka
než
úsečka
PQ
.
být
r
bod
ležící
na
úsečka
page
takový
,
že
úsečka
RP
,
QQ
mít
stejný
délka
(
viz
obrázek
.
19
)
.
být
RN
přímka
takový
,
že
úhel
NRP
být
stejně
veliký
jako
úhel
SQQ
,
kde
n
být
bod
ležící
v
týž
polorovina
určený
přímka
page
jako
bod
q
.
být
QM
souběžka
přímka
p
,
přičemž
směr
souběžnost
tento
přímka
být
směr
od
bod
q
k
bod
m
a
od
bod
q
k
bod
s
.
být
h
takový
bod
,
že
úhel
HPQ
,
MQS
být
stejně
veliký
a
bod
h
ležet
v
týž
polorovina
určený
přímka
p
jako
bod
m
.
podle
tvrzení
2
být
přímka
QM
,
PH
rozběžka
.
přímka
RN
,
p
se
protnout
v
nějaký
bod
a
,
neboť
v
opačný
případ
být
souběžka
přímka
p
vedený
bod
q
svírat
s
přímka
q
velký
úhel
než
přímka
QM
,
ten
být
alespoň
takový
úhel
,
jaký
s

,
ráz
on
zařadit
mezi
podivný
z
ten
,
který
být
až
dosud
o
můj
odsouzený
geometrický
svět
dokázat
.
tak
například
ať
zvolit
jakkoliv
malý
úhel
,
pak
vždy
uvnitř
tento
úhel
lze
vést
přímka
,
který
nikdy
protnout
jeho
rameno
.
stačit
totiž
vést
přímka
kolmý
na
osa
tento
úhel
v
vzdálenost
od
vrchol
,
který
být
stejně
velký
,
nebo
velký
než
vzdálenost
bod
p
od
přímka
p
,
jenž
úhel
souběžnost
s
přímka
p
být
rovný
polovina
tento
zvolený
úhel
.
vytvářet
-
li
pravoúhlý
rovnoramenný
trojúhelník
o
stále
velký
rozměr
,
pak
délka
jeho
výška
být
shora
omezený
délka
k
,
kde
k
být
vzdálenost
bod
p
od
přímka
p
,
jenž
úhel
souběžnost
s
tento
přímka
být
rovný
polovina
úhel
pravý
.
zvolit
-
li
dva
kolmý
přímka
p
,
q
protínající
se
v
bod
s
a
vést
vždy
v
daný
vzdálenost
od
bod
s
přímka
kolmý
na
tento
přímka
,
být
zprvu
vznikat
čtyřúhelník
o
stejně
dlouhý
strana
.
jakmile
však
tento
vzdálenost
dosáhnout
brzy
uvedený
délka
k
,
utéci
vrchol
tento
čtyřúhelník
do
nekonečno
,
a
tedy
jeho
"
sousední
"
strana
se
n

před
obzor
.
tento
úvaha
být
však
správný
a
odhalení
omyl
v
on
obsažený
být
poučný
.
když
být
se
totiž
jednou
rozhodnout
vykládat
světelný
paprsek
jako
přímka
,
pak
se
muset
tento
výklad
držet
důsledně
.
i
paprsek
vycházející
z
můj
oko
muset
tedy
podřídit
zákon
prostor
můj
odsouzený
geometrický
svět
.
být
být
žádný
umění
dospět
k
spor
,
_
když
být
být
během
týž
úvaha
vykládat
chování
světelný
paprsek
různý
způsob
.
_
aby
být
mít
bod
p
a
přímka
p
,
c
,
d
před
se
jako
na
dlaň
,
muset
se
dívat
na
rovina
r
z
vzdálenost
velký
než
parametr
.
ten
ale
znamenat
,
že
rovina
r
protínat
pouze
ten
paprsek
vycházející
z
můj
oko
,
který
ležet
uvnitř
světelný
kužel
,
jenž
povrchový
přímka
svírat
s
kolmice
na
rovina
r
úhel
stejně
veliký
jako
úhel
souběžnost
bod
,
v
jenž
se
nacházet
můj
oko
,
s
rovina
r
.
dívat
-
li
se
po
přímka
,
který
ležet
uvnitř
tento
kužel
,
pak
se
na
rovina
r
dívat
,
neboť
tento
přímka
rovina
r
protnout
.
celý
rovina
r
se
já
tedy
jevit
jako
kruh
;
co
ležet
uvnitř
tento
kruh
,
ten


Считаем ошибку как отношение количества неправильно предсказанных тегов к общему числу тегов

In [87]:
err = 0.
for sent in range(len(test_tags)):
    err += np.mean(np.array(test_tags[sent]) != np.array(predict_tags[sent]))
print('Error = ', err/len(test_tags))   

Error =  0.05380520276089736
